In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from typing import List, Tuple
import matplotlib.pyplot as plt
import numpy as np
import loompy
import shoji
from tqdm import trange
import logging
import sys
%load_ext line_profiler
logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=20)

In [2]:
db = shoji.connect()
db

,Contents
eel,"2 workspaces, 0 dimensions, 0 tensors"
images,"0 workspaces, 1 dimensions, 1 tensors"
refdb,"2 workspaces, 0 dimensions, 0 tensors"
scRNA,"0 workspaces, 2 dimensions, 3 tensors"
test,"0 workspaces, 2 dimensions, 42 tensors"


In [3]:
if "human_development" in db.refdb:
    del db.refdb.human_development
db.refdb.human_development = shoji.Workspace()

In [4]:
ws = db.refdb.human_development
ws._from_loom("/Users/stelin/Allbrain.agg.loom", dimension_names=("genes", "clusters"))

100%|██████████| 1/1 [03:47<00:00, 227.85s/it]


In [5]:
ws

Workspace with 0 workspaces, 2 dimensions and 375 tensors:
  clusters <Dimension of variable shape>
  genes <Dimension of shape 33538>
  Accession <Tensor Accession dtype='string' dims=('genes',), shape=(33538,)>
  Age_10w <Tensor Age_10w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_11_5w <Tensor Age_11_5w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_12w <Tensor Age_12w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_13-13_5w <Tensor Age_13-13_5w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_14w <Tensor Age_14w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_5_5w <Tensor Age_5_5w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_5w <Tensor Age_5w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_6_6w <Tensor Age_6_6w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_6_7w <Tensor Age_6_7w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_6_9w <Tensor Age_6_9w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_6w <Tensor Age_6w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_7_5w <Tensor Age_7_5w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_8_1w <Tensor Age_8_1w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_8_5w <Tensor Age_8_5w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_8w <Tensor Age_8w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_9_2w <Tensor Age_9_2w dtype='int64' dims=('clusters',), shape=(616,)>
  Age_9_5w <Tensor Age_9_5w dtype='int64' dims=('clusters',), shape=(616,)>
  AutoAnnotation <Tensor AutoAnnotation dtype='string' dims=('clusters',), shape=(616,)>
  Chromosome <Tensor Chromosome dtype='string' dims=('genes',), shape=(33538,)>
  Clusters <Tensor Clusters dtype='int64' dims=('clusters',), shape=(616,)>
  Config <Tensor Config dtype='string' dims=(), shape=()>
  CreationDate <Tensor CreationDate dtype='string' dims=(), shape=()>
  End <Tensor End dtype='int64' dims=('genes',), shape=(33538,)>
  Enrichment <Tensor Enrichment dtype='float64' dims=('clusters', 'genes'), shape=(616, 33538)>
  Expression <Tensor Expression dtype='float64' dims=('clusters', 'genes'), shape=(616, 33538)>
  Gene <Tensor Gene dtype='string' dims=('genes',), shape=(33538,)>
  LOOM_SPEC_VERSION <Tensor LOOM_SPEC_VERSION dtype='string' dims=(), shape=()>
  Linkage <Tensor Linkage dtype='float64' dims=(615, 4), shape=(615, 4)>
  MarkerGenes <Tensor MarkerGenes dtype='string' dims=('clusters',), shape=(616,)>
  MarkerRobustness <Tensor MarkerRobustness dtype='string' dims=('clusters',), shape=(616,)>
  MarkerSelectivity <Tensor MarkerSelectivity dtype='string' dims=('clusters',), shape=(616,)>
  MarkerSpecificity <Tensor MarkerSpecificity dtype='string' dims=('clusters',), shape=(616,)>
  MultilevelMarkers <Tensor MultilevelMarkers dtype='float64' dims=('genes', 4), shape=(33538, 4)>
  NCells <Tensor NCells dtype='int64' dims=('clusters',), shape=(616,)>
  Outliers <Tensor Outliers dtype='float64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_1 <Tensor SampleID_10X101_1 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_2 <Tensor SampleID_10X101_2 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_3 <Tensor SampleID_10X101_3 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_4 <Tensor SampleID_10X101_4 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_5 <Tensor SampleID_10X101_5 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_6 <Tensor SampleID_10X101_6 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_7 <Tensor SampleID_10X101_7 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X101_8 <Tensor SampleID_10X101_8 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X102_1 <Tensor SampleID_10X102_1 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X102_2 <Tensor SampleID_10X102_2 dtype='int64' dims=('clusters',), shape=(616,)>
  SampleID_10X102_3 <Tensor SampleID_10X102_3 dtype='int64' dims=('clusters',), shape=(616,)>
  S

In [23]:
import scipy.cluster.hierarchy as hc
ws = db.refdb.human_development


def enrichment(ws: shoji.WorkspaceManager) -> np.ndarray:
    """
    Compute gene enrichment for a workspace of aggregate data, by cluster

    Args:
        ws		The workspace

    Returns:
        enrichment		A (n_clusters, n_genes) matrix of gene enrichment scores
    """
    n_clusters = len(ws.clusters)
    view = ws[:]

    totals = view.Expression.sum(axis=1)
    x_norm = (view.Expression.T / totals * np.median(totals)).T
    gene_sums = x_norm.sum(axis=0)
    enrichment = np.zeros_like(x_norm)
    for j in range(n_clusters):
        enrichment[j, :] = (x_norm[j, :] + 1) / (((gene_sums - x_norm[j, :]) / n_clusters) + 1)
    return enrichment


def enrichment_by_groups(ws: shoji.WorkspaceManager, labels: np.ndarray) -> np.ndarray:
    """
    Compute gene enrichment for a workspace of aggregate data, by groups of clusters

    Args:
        ws		The workspace
        labels	Labels (0, 1, ...) indicating the cluster groups

    Returns:
        enrichment		A (n_groups, n_genes) matrix of gene enrichment scores
    """
    n_clusters = labels.max() + 1
    view = ws[:]
    data = (view.Expression.T * view.NCells).T
    grouped = np.zeros((n_clusters, data.shape[1]))
    for j in range(n_clusters):
        grouped[j, :] = data[labels == j, :].sum(axis=0) / view.NCells[labels == j].sum()

    totals = grouped.sum(axis=1)
    x_norm = (grouped.T / totals * np.median(totals)).T
    gene_sums = x_norm.sum(axis=0)
    enrichment = np.zeros_like(x_norm)
    for j in range(n_clusters):
        enrichment[j, :] = (x_norm[j, :] + 1) / (((gene_sums - x_norm[j, :]) / n_clusters) + 1)
    return enrichment


def multilevel_feature_selection(ws: shoji.WorkspaceManager, preselected=[]):
    n_clusters = len(ws.clusters)

    n = 2
    selected: List[int] = []
    genes = ws[:].Gene
    for gene in preselected:
        if gene in genes:
            selected.append(np.where(genes == gene)[0][0])

    # Select from the dendrogram
    while n < n_clusters // 2:
        labels = hc.cut_tree(ws[:].Linkage, n_clusters=n).T[0]
        enr = enrichment_by_groups(ws, labels)
        for j in range(n):
            top = np.argsort(-enr[j, :])
            for t in top:
                if t not in selected:
                    selected.append(t)
                    break
        n *= 2

    # Select from the leaves
    enr = enrichment(ws)
    for j in range(n_clusters):
        top = np.argsort(-enr[j, :])
        for t in top:
            if t not in selected:
                selected.append(t)
                break
    return selected


In [24]:
enr = multilevel_feature_selection(ws, preselected=["PNMT",
"AQP4",
"GJA1",
"CD4",
"IL7R",
"TRAC",
"CCL5",
"CD8",
"GZMA",
"GZMK",
"CDK1",
"PCNA",
"TOP2A",
"UBE2C",
"CHAT",
"SLC18A3",
"SLC5A7",
"CST3",
"HLA-DBP1",
"HLA-DQB1","SLC6A3",
"TH",
"CLDN5",
"FLT1",
"VWF",
"FOXJ1",
"TMEM212",
"COL1A1",
"COL3A1",
"DCN",
"LOX",
"LUM",
"GAD1",
"GAD2",
"SLC32A1",
"SLC17A6",
"SLC17A7",
"SLC17A8",
"SLC6A5",
"SLC6A9",
"ACSL1",
"APOE",
"C1QA",
"C1QB",
"C1QC",
"CCL18",
"CCL20",
"CCL3",
"CCL3L1",
"CCL4",
"CCL4L2",
"CD163",
"CD84",
"CD93",
"CSF2RA",
"CTSB",
"CXCL8",
"CYBB",
"DAB2",
"EGR3",
"EMB",
"EREG",
"FCGBP",
"FCGR3A",
"FGL1",
"FOLR2",
"GPNMB",
"GPR183",
"GPR34",
"HBG1",
"HK2",
"HMOX1",
"IBSP",
"IL1B",
"IL1R2",
"KYNU",
"LSP1",
"MSR1",
"NAV3",
"OLR1",
"OSM",
"P2RY12",
"PPBP",
"RAB20",
"S100A4",
"S100A8",
"S100A9",
"SCML4",
"SLAMF8",
"SLC02B1",
"SLC16A10",
"SPP1",
"STAB1",
"TFEC",
"TGFBI",
"TMIGD3",
"TREM2",
"TYROBP",
"VAMP8",
"VSIG4",
"AIF1",
"HEXB",
"SLC18A1",
"SLC18A2",
"CD36",
"CD74",
"IFI30",
"LYZ",
"PSAP",
"CD79B",
"MS4A1",
"EBF1",
"EBF2",
"EBF3",
"NHLH1",
"NHLH2",
"RBFOX3",
"STMN2",
"STMN3",
"GNLY",
"NKG7",
"DBH",
"MBP",
"MOG",
"PLP1",
"SOX10",
"PDGFRA",
"MRC1",
"MZB1",
"TXNDC5",
"IRF8",
"TCF4",
"FABP5",
"FABP7",
"HOPX",
"PAX6",
"MPZ",
"FEV",
"TPH2",
"CD2",
"CD3D",
"CD69",
"GZMB",
"IGKC",
"JCHAIN",
"KLRB1",
"LTB",
"TRBC2",
"HMGB2",
"STMN1",
"TUBA1B",
"FOXP3",
"IL32",
"ACTA2",
"TAGLN"])

In [27]:
for g in sorted(ws[:].Gene[enr[152+287:152+287+100]]):
    print(g)

AC092691.1
BHLHE22
CADM1
CADPS2
CARTPT
CCNB2
CDH18
CDK14
CHRM3
CLYBL
CNTN1
CNTN4
CNTNAP4
COL25A1
CRH
DBI
DCLK2
DLGAP1
DNAJC12
DSCAML1
EN1
EPHA5
EPHA6
FAM19A1
FOXD3-AS1
FSTL5
GAP43
GBX1
GPC6
GRIA2
GRIP1
GRM5
HDC
HOTAIRM1
INPP4B
ISL1
KAZN
KCNQ3
KIF26B-AS1
KIZ
KLHL1
LDB2
LHFPL3
LHFPL6
LINC02381
LINGO2
LMO1
LUZP2
MDGA2
MEF2C
MEST
MSRA
NCAM2
NDST3
NEGR1
NEUROD2
NKX2-1
NLGN1
NNAT
NPY
NR2F2
NR2F2-AS1
NRCAM
NRXN1
NTM
NTRK3
PCSK1N
PDE1A
PDE4D
PPP2R2B
PRKCA
PRKCB
PTPRG
SCGN
SEMA6D
SGCZ
SHISA6
SHTN1
SIM1
SIRT2
SIX3
SLIT3
SMC4
SNTG1
SPOCK1
SSTR2
ST18
SYN3
TBCA
TENM4
THRB
TSHZ3
TUBA1A
UBE2S
UCHL1
VCAN
ZIC1
ZNF804A
ZNF804B
ZSWIM5
